In [1]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lg
from tqdm import tqdm
import pandas as pd
from pathlib import Path

# from importlib import reload


import catboost
from catboost.utils import get_gpu_device_count

sys.path.append("../../scorer/")
import orderbook_fast as ob_fast

# from  solution import get_simple_features_from_orderbook, get_simple_deals_features
# from  solution import get_simple_deals_features

from my_orderbook import MyOrderBook, action_handler
catboost_myob = MyOrderBook()

SIDE_BID = 0 
SIDE_ASK = 1

print(f"curdir: {Path.cwd()}")

# #%% md

## Собираем датасет для тренировки модели

curdir: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/examples/catboost_local


In [2]:
def collect_dataset(data_path):
    '''
        Собираем датасет
    '''

    global catboost_myob
    event_player = ob_fast.EventPlayer(data_path)
    orderbook_fast = ob_fast.OrderBook()

    X = []
    Y = []

    for ev in tqdm(event_player.iget_events(),
                    total=len(event_player),
                    desc=f"collecting dataset {data_path}"):

        catboost_myob.__getattribute__(action_handler[ev.action])(ev)

        if ev.need_prediction:
            features = catboost_myob.get_features(ev)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return pd.DataFrame(X), pd.DataFrame(Y)

# X_train_very_small_A, Y_train_very_small_A = collect_dataset("../../data/very_small_A.npz")
# X_train_very_small_A.to_pickle("../../data/X_train_very_small_A_130.pickle")
# Y_train_very_small_A.to_pickle("../../data/Y_train_very_small_A_130.pickle")

# X_test_very_small_B, Y_test_very_small_B = collect_dataset("../../data/very_small_B.npz")
# X_test_very_small_B.to_pickle("../../data/X_test_very_small_B_130.pickle")
# Y_test_very_small_B.to_pickle("../../data/Y_test_very_small_B_130.pickle")

X_train, Y_train = collect_dataset("../../data/X_train_small_A.npz")
X_train.to_pickle("../../data/X_train.pickle")
Y_train.to_pickle("../../data/Y_train.pickle")

X_test, Y_test = collect_dataset("../../data/X_train_small_B.npz")
X_test.to_pickle("../../data/X_test.pickle")
Y_test.to_pickle("../../data/Y_test.pickle")

print( "********* FINISH dataset collection ***********************")

collecting dataset ../../data/X_train_small_A.npz:   0%|          | 1/767493 [00:44<9409:58:01, 44.14s/it]


KeyboardInterrupt: 